In [1]:
import pandas as pd

import re

In [2]:
DATA_PATH="../"

In [3]:
# Format for Federal Law LexML URN:
#
#    https://normas.leg.br/?urn=urn:lex:br:federal:lei:{<year>}-{<month>}-{<day>};{<law-number}
#

LEXML_FEDERAL_LAW_URN_FORMAT="https://normas.leg.br/?urn=urn:lex:br:federal:lei:{}-{:02}-{:02};{}"

In [4]:
# Month converter

MONTHS={
    "janeiro": 1,
    "fevereiro": 2,
    "março": 3,
    "abril": 4,
    "maio": 5,
    "junho": 6,
    "julho": 7,
    "agosto": 8,
    "setembro": 9,
    "outubro": 10,
    "novembro": 11,
    "dezembro": 12    
}

In [5]:
references_filename_map_df = pd.read_csv(DATA_PATH + "reference_filename_map.tsv", 
                                         sep="\t", 
                                         index_col=0)

In [6]:
all_references = sorted(references_filename_map_df.index)

In [7]:
legal_rules_references = [ref for ref in all_references if ref.startswith("Lei") or ref.startswith("lei")]

In [8]:
legal_rules_references

['Lei Complementar nº 109, de 29 de maio de 2001',
 'Lei Complementar nº 123, de 14 de dezembro de 2006',
 'Lei n° 10.406, de 10 de janeiro de 2002 - Código Civil',
 'Lei n° 10.522, de 19 de julho de 2002',
 'Lei nº 10.101, de 19 de dezembro de 2000',
 'Lei nº 10.200, de 14 de fevereiro de 2001',
 'Lei nº 10.406, de 10 de janeiro de 2002 - Código Civil',
 'Lei nº 10.451, de 10 de maio de 2002',
 'Lei nº 10.522, de 19 de julho de 2002',
 'Lei nº 10.559, de 13 de novembro de 2002',
 'Lei nº 10.698, de 2 de julho de 2003',
 'Lei nº 10.741, de 1º de outubro de 2003',
 'Lei nº 10.833, de 29 de dezembro de 2003',
 'Lei nº 10.865, de 30 de abril de 2004',
 'Lei nº 10.973, de 2 de dezembro de 2004',
 'Lei nº 11.033, de 21 de dezembro de 2004',
 'Lei nº 11.051, de 29 de dezembro de 2004',
 'Lei nº 11.053, de 29 de dezembro de 2004',
 'Lei nº 11.196, de 21 de novembro de 2005',
 'Lei nº 11.312, de 27 de junho de 2006',
 'Lei nº 11.324, de 19 de julho de 2006',
 'Lei nº 11.437, de 28 de dezembro 

## Some manual fixes

In [9]:
legal_rules_references = legal_rules_references[4:]

In [10]:
legal_rules_references.index('Lei nº 4.242, de 1963')

55

In [11]:
legal_rules_references[55] = 'Lei nº 4.242, de 17 de julho de 1963'

In [12]:
legal_rules_references.index('Lei nº 10.741, de 1º de outubro de 2003')

7

In [13]:
legal_rules_references[7] = 'Lei nº 10.741, de 1 de outubro de 2003'

In [14]:
legal_rules_references

['Lei nº 10.101, de 19 de dezembro de 2000',
 'Lei nº 10.200, de 14 de fevereiro de 2001',
 'Lei nº 10.406, de 10 de janeiro de 2002 - Código Civil',
 'Lei nº 10.451, de 10 de maio de 2002',
 'Lei nº 10.522, de 19 de julho de 2002',
 'Lei nº 10.559, de 13 de novembro de 2002',
 'Lei nº 10.698, de 2 de julho de 2003',
 'Lei nº 10.741, de 1 de outubro de 2003',
 'Lei nº 10.833, de 29 de dezembro de 2003',
 'Lei nº 10.865, de 30 de abril de 2004',
 'Lei nº 10.973, de 2 de dezembro de 2004',
 'Lei nº 11.033, de 21 de dezembro de 2004',
 'Lei nº 11.051, de 29 de dezembro de 2004',
 'Lei nº 11.053, de 29 de dezembro de 2004',
 'Lei nº 11.196, de 21 de novembro de 2005',
 'Lei nº 11.312, de 27 de junho de 2006',
 'Lei nº 11.324, de 19 de julho de 2006',
 'Lei nº 11.437, de 28 de dezembro de 2006',
 'Lei nº 11.438, de 29 de dezembro de 2006',
 'Lei nº 11.472, de 2 de maio de 2007',
 'Lei nº 11.482, de 31 de maio de 2007',
 'Lei nº 11.524, de 24 de setembro de 2007',
 'Lei nº 11.727, de 23 de j

In [15]:
all_legal_norms = []

for title in legal_rules_references:
    # m = re.match("Lei[^0-9]+([0-9\.]+)\,\sde\s([0-9]+)\s(\w+)\s([0-9])+", title)
    m = re.match("Lei[^0-9]+([0-9\.]+),\sde\s([0-9]+)\sde\s(\w+)\sde\s([0-9]+)", title)
    print("{}: {}-{}-{};{}".format(title, 
                                   m.group(4), 
                                   m.group(3),
                                   m.group(2),
                                   m.group(1).replace(".", "")))

    all_legal_norms.append(LEXML_FEDERAL_LAW_URN_FORMAT.format(m.group(4), 
                                                               int(MONTHS[m.group(3).lower()]),
                                                               int(m.group(2)),
                                                               m.group(1).replace(".", "")))

Lei nº 10.101, de 19 de dezembro de 2000: 2000-dezembro-19;10101
Lei nº 10.200, de 14 de fevereiro de 2001: 2001-fevereiro-14;10200
Lei nº 10.406, de 10 de janeiro de 2002 - Código Civil: 2002-janeiro-10;10406
Lei nº 10.451, de 10 de maio de 2002: 2002-maio-10;10451
Lei nº 10.522, de 19 de julho de 2002: 2002-julho-19;10522
Lei nº 10.559, de 13 de novembro de 2002: 2002-novembro-13;10559
Lei nº 10.698, de 2 de julho de 2003: 2003-julho-2;10698
Lei nº 10.741, de 1 de outubro de 2003: 2003-outubro-1;10741
Lei nº 10.833, de 29 de dezembro de 2003: 2003-dezembro-29;10833
Lei nº 10.865, de 30 de abril de 2004: 2004-abril-30;10865
Lei nº 10.973, de 2 de dezembro de 2004: 2004-dezembro-2;10973
Lei nº 11.033, de 21 de dezembro de 2004: 2004-dezembro-21;11033
Lei nº 11.051, de 29 de dezembro de 2004: 2004-dezembro-29;11051
Lei nº 11.053, de 29 de dezembro de 2004: 2004-dezembro-29;11053
Lei nº 11.196, de 21 de novembro de 2005: 2005-novembro-21;11196
Lei nº 11.312, de 27 de junho de 2006: 2006-

In [16]:
all_legal_norms

['https://normas.leg.br/?urn=urn:lex:br:federal:lei:2000-12-19;10101',
 'https://normas.leg.br/?urn=urn:lex:br:federal:lei:2001-02-14;10200',
 'https://normas.leg.br/?urn=urn:lex:br:federal:lei:2002-01-10;10406',
 'https://normas.leg.br/?urn=urn:lex:br:federal:lei:2002-05-10;10451',
 'https://normas.leg.br/?urn=urn:lex:br:federal:lei:2002-07-19;10522',
 'https://normas.leg.br/?urn=urn:lex:br:federal:lei:2002-11-13;10559',
 'https://normas.leg.br/?urn=urn:lex:br:federal:lei:2003-07-02;10698',
 'https://normas.leg.br/?urn=urn:lex:br:federal:lei:2003-10-01;10741',
 'https://normas.leg.br/?urn=urn:lex:br:federal:lei:2003-12-29;10833',
 'https://normas.leg.br/?urn=urn:lex:br:federal:lei:2004-04-30;10865',
 'https://normas.leg.br/?urn=urn:lex:br:federal:lei:2004-12-02;10973',
 'https://normas.leg.br/?urn=urn:lex:br:federal:lei:2004-12-21;11033',
 'https://normas.leg.br/?urn=urn:lex:br:federal:lei:2004-12-29;11051',
 'https://normas.leg.br/?urn=urn:lex:br:federal:lei:2004-12-29;11053',
 'http

## Save the law URNs

In [17]:
with open("legal_documents/law_rules_URNs.txt", "w") as output_file:
    for urn in all_legal_norms[:-1]:
        output_file.write(urn + "\n")

    output_file.write(all_legal_norms[-1])

## Using the Legal Document Fetcher Module

Now we'll use the Object-Oriented module to fetch all legal documents and save them as Word files.

In [18]:
# Import the module
from legal_document_fetcher import LegalDocumentFetcher, FetcherConfig

In [19]:
# Configure the fetcher
config = FetcherConfig(
    output_dir='./legal_documents',
    retry_attempts=3,
    delay_between_requests=2.0
)

print(f"Output directory: {config.output_dir}")
print(f"Timeout: {config.request_timeout}s")
print(f"Retry attempts: {config.retry_attempts}")
print(f"Delay between requests: {config.delay_between_requests}s")

Output directory: ./legal_documents
Timeout: 30s
Retry attempts: 3
Delay between requests: 2.0s


In [20]:
# Create the fetcher instance
fetcher = LegalDocumentFetcher(config)

In [21]:
# Test with a single URL first
test_url = all_legal_norms[0]
print(f"Testing with: {test_url}")

test_result = fetcher.process_single_url(test_url)
print(f"\nResult: {test_result}")

Testing with: https://normas.leg.br/?urn=urn:lex:br:federal:lei:2000-12-19;10101


2025-11-23 23:47:18,873 - legal_document_fetcher - INFO - Selenium WebDriver initialized successfully
2025-11-23 23:47:18,874 - legal_document_fetcher - INFO - Fetching with Selenium: https://normas.leg.br/?urn=urn:lex:br:federal:lei:2000-12-19;10101
2025-11-23 23:47:19,767 - legal_document_fetcher - INFO - Found sf-unstructured-legislation-viewer element
2025-11-23 23:47:19,781 - legal_document_fetcher - INFO - Extracted 105357 bytes from shadow DOM
2025-11-23 23:47:19,791 - legal_document_fetcher - INFO - Successfully fetched 243465 bytes with Selenium
2025-11-23 23:47:19,805 - legal_document_fetcher - INFO - Detected shadow DOM content marker
2025-11-23 23:47:19,815 - legal_document_fetcher - INFO - Found main content div in shadow DOM
2025-11-23 23:47:19,827 - legal_document_fetcher - INFO - Skipping title heading (generic or too long): Lei: Art. 1º Esta Lei regula a participação dos trabalhadores nos lucros ou resultados da empresa co...
2025-11-23 23:47:19,828 - legal_document_fe


Result: ✓ lei_10101_20001219 -> lei_10101_20001219.docx (1.49s)


In [22]:
# Process all URLs
# WARNING: This will take a while (about 3-4 minutes for 108 URLs with 2s delay)
# Uncomment the line below to run:

results = fetcher.process_url_list(all_legal_norms, show_progress=True)

Fetching documents:   0%|                                                                                                                                                            | 0/106 [00:00<?, ?it/s]2025-11-23 23:47:56,052 - legal_document_fetcher - INFO - Fetching with Selenium: https://normas.leg.br/?urn=urn:lex:br:federal:lei:2000-12-19;10101
2025-11-23 23:48:16,625 - legal_document_fetcher - INFO - Shadow DOM element not found or timeout: Message: 
Stacktrace:
#0 0x632a41e814ca <unknown>
#1 0x632a418cfe4b <unknown>
#2 0x632a41922814 <unknown>
#3 0x632a41922a51 <unknown>
#4 0x632a41971a44 <unknown>
#5 0x632a4196edf7 <unknown>
#6 0x632a41914c2a <unknown>
#7 0x632a41915931 <unknown>
#8 0x632a41e47cf9 <unknown>
#9 0x632a41e4acdc <unknown>
#10 0x632a41e30f79 <unknown>
#11 0x632a41e4b8b5 <unknown>
#12 0x632a41e189c3 <unknown>
#13 0x632a41e6e228 <unknown>
#14 0x632a41e6e403 <unknown>
#15 0x632a41e80463 <unknown>
#16 0x7df5fee6bac3 <unknown>

2025-11-23 23:48:16,626 - legal_document_

In [23]:
# View summary statistics
summary = fetcher.get_summary()

print("=" * 60)
print("FETCH SUMMARY")
print("=" * 60)
print(f"Total URLs processed:    {summary['total']}")
print(f"Successful:              {summary['success']}")
print(f"Failed:                  {summary['failed']}")
print(f"Success rate:            {summary['success_rate']:.2f}%")
print(f"Average fetch time:      {summary['avg_fetch_time']:.2f}s")
print("=" * 60)

if summary['failed_urls']:
    print(f"\nFailed URLs ({len(summary['failed_urls'])}):")
    for url in summary['failed_urls']:
        print(f"  - {url}")

FETCH SUMMARY
Total URLs processed:    106
Successful:              96
Failed:                  10
Success rate:            90.57%
Average fetch time:      3.24s

Failed URLs (10):
  - https://normas.leg.br/?urn=urn:lex:br:federal:lei:2000-12-19;10101
  - https://normas.leg.br/?urn=urn:lex:br:federal:lei:2007-09-24;11524
  - https://normas.leg.br/?urn=urn:lex:br:federal:lei:2008-10-08;11795
  - https://normas.leg.br/?urn=urn:lex:br:federal:lei:2023-12-12;14754
  - https://normas.leg.br/?urn=urn:lex:br:federal:lei:1966-10-25;5172
  - https://normas.leg.br/?urn=urn:lex:br:federal:lei:1990-12-11;8112
  - https://normas.leg.br/?urn=urn:lex:br:federal:lei:1991-12-30;8383
  - https://normas.leg.br/?urn=urn:lex:br:federal:lei:1995-01-20;8891
  - https://normas.leg.br/?urn=urn:lex:br:federal:lei:1996-05-10;9278
  - https://normas.leg.br/?urn=urn:lex:br:federal:lei:1998-03-24;9615


In [24]:
# Export results to CSV for analysis
fetcher.export_results_to_csv('fetch_results.csv')
print("Results exported to fetch_results.csv")

2025-11-24 00:03:41,109 - legal_document_fetcher - INFO - Results exported to fetch_results.csv


Results exported to fetch_results.csv
